In [176]:
import numpy as np
import pandas as pd
df = pd.read_csv("data (1).csv")
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
pd.options.mode.chained_assignment = None
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [177]:
df.loc[(df.days_employed < 0), 'days_employed'] = df.days_employed.abs()
df.fillna(df['days_employed'].median(), inplace=True)

In [178]:
df[['total_income']] = df[['total_income']].astype(int)
df[['days_employed']] = df[['days_employed']].astype(int)
df = df.drop_duplicates()
df['education'] = df['education'].str.capitalize()

In [179]:
dfED = pd.DataFrame({'education': df['education'].drop_duplicates(), 'education_id':  df['education_id'].drop_duplicates()})
dfED

,education,education_id
0,Высшее,0
1,Среднее,1
13,Неоконченное высшее,2
31,Начальное,3
2963,Ученая степень,4


In [180]:
dfFAM_STAT = pd.DataFrame({'family_status': df['family_status'].drop_duplicates(), 'family_status_id':  df['family_status_id'].drop_duplicates()})
dfFAM_STAT

,family_status,family_status_id
0,женат / замужем,0
4,гражданский брак,1
18,вдовец / вдова,2
19,в разводе,3
24,Не женат / не замужем,4


In [181]:
def tot_inc_cat(row):
    if row['total_income'] <= 30000:
        return "E"
    elif row['total_income'] >= 30001 and row['total_income'] <= 50000:
        return "D"
    elif row['total_income'] >= 50001 and row['total_income'] <= 200000:
        return "C"
    elif row['total_income'] >= 200001 and row['total_income'] <= 1000000:
        return "B"
    else:
        return "A"
    
df['total_income_category'] = df.apply(tot_inc_cat, axis=1)

def purp_cat(row):
    if 'авто' in row['purpose']:
        return 'Операции с автомобилем'
    elif 'недвиж' in row['purpose'] or 'жил' in row['purpose']:
        return 'Операции с недвижимостью'
    elif 'свадьб' in row['purpose']:
        return 'Проведение свадьбы'
    else:
        return 'Получение образования'

df['purpose_category'] = df.apply(purp_cat, axis=1)

Приводим дата фрейм к нормальному виду исходя из опыта прошлого занятия

In [182]:
df = df.drop(columns=['education','family_status', 'total_income', 'purpose'])
df

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,B,Операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,C,Операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,C,Операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,B,Получение образования
4,0,340266,53,1,1,F,пенсионер,0,C,Проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,1,1,F,компаньон,0,B,Операции с недвижимостью
21521,0,343937,67,1,0,F,пенсионер,0,C,Операции с автомобилем
21522,1,2113,38,1,1,M,сотрудник,1,C,Операции с недвижимостью
21523,3,3112,38,1,0,M,сотрудник,1,B,Операции с автомобилем


In [183]:
X = df.to_numpy()[:, (0,1,2,3,4)]
y = df['debt']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=52)

In [184]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

C:\Users\musin\PycharmProjects\MAI_BIGdata_PZ2\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [185]:
training_prediction = log_reg.predict(X_train)
log_reg.score(X_train, y_train)

0.9192361434559851

Рассчитаем значение accuracy для тренировочного набора данных

In [186]:
from sklearn import metrics
print (metrics.classification_report(y_train, training_prediction, digits=5))

              precision    recall  f1-score   support

           0    0.91924   1.00000   0.95792      9868
           1    0.00000   0.00000   0.00000       867

    accuracy                        0.91924     10735
   macro avg    0.45962   0.50000   0.47896     10735
weighted avg    0.84500   0.91924   0.88055     10735


C:\Users\musin\PycharmProjects\MAI_BIGdata_PZ2\venv\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\musin\PycharmProjects\MAI_BIGdata_PZ2\venv\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\musin\PycharmProjects\MAI_BIGdata_PZ2\venv\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

Используя встроенную функцию библиотеки также рассчитаем значение accuracy с округением до 5 знака после запятой. Как видим, значения совпали и оба держатся выше отметки 91%, что является очень хорошим результатом. Теперь приступим к обучению модели методом случайного леса

In [187]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=310, n_estimators=5)

In [188]:
A = df.to_numpy()[:, (0,4)]
B = df['debt']
A_train, A_test, B_train, B_test = train_test_split(A, B, test_size=0.25, random_state=310)

In [189]:
model.fit(A_train, B_train)

RandomForestClassifier(n_estimators=5, random_state=310)

In [190]:
r_s = 310
accuracy_forest_best = None
for estim in range (1, 102, 5):
    model2 = RandomForestClassifier(n_estimators=estim, max_depth=9, random_state=r_s)
    model2.fit(A_train, B_train)
    model2TrainingPred = model2.predict(A_test)
    accuracyForest = model2.score(A_train, B_train)
    print("n_estimators = ", estim, ":", end='')
    if accuracy_forest_best is None or accuracyForest > accuracy_forest_best:
        accuracy_forest_best = accuracyForest
        n_estimators_best = estim
    print('max_depth_best = ',n_estimators_best, "accuracy_tree_best: ", accuracy_forest_best)


n_estimators =  1 :max_depth_best =  1 accuracy_tree_best:  0.9195802024467491
n_estimators =  6 :max_depth_best =  6 accuracy_tree_best:  0.9196423026765199
n_estimators =  11 :max_depth_best =  6 accuracy_tree_best:  0.9196423026765199
n_estimators =  16 :max_depth_best =  6 accuracy_tree_best:  0.9196423026765199
n_estimators =  21 :max_depth_best =  6 accuracy_tree_best:  0.9196423026765199
n_estimators =  26 :max_depth_best =  6 accuracy_tree_best:  0.9196423026765199
n_estimators =  31 :max_depth_best =  6 accuracy_tree_best:  0.9196423026765199
n_estimators =  36 :max_depth_best =  6 accuracy_tree_best:  0.9196423026765199
n_estimators =  41 :max_depth_best =  6 accuracy_tree_best:  0.9196423026765199
n_estimators =  46 :max_depth_best =  6 accuracy_tree_best:  0.9196423026765199
n_estimators =  51 :max_depth_best =  6 accuracy_tree_best:  0.9196423026765199
n_estimators =  56 :max_depth_best =  6 accuracy_tree_best:  0.9196423026765199
n_estimators =  61 :max_depth_best =  6 ac

Значение accuracy приблизительно равно 92%, что является очень неплохим результатом